In [7]:
import pandas as pd
import numpy as np
import pickle
from tqdm import tqdm
import warnings
# cpu multitasking
import dask
from dask.diagnostics import ProgressBar
import os





#from data_preprocess import dataloader_preproceser, same_day_only_one,  minmax_normalize 
######################################### creating global variables




############## helpers

def same_day_only_one(sdc_tnic_raw):
    print("shape before removing same-day multi events:", sdc_tnic_raw.shape)
    sdc_tnic_raw = sdc_tnic_raw.copy()
    sdc_tnic_one = sdc_tnic_raw.groupby(['AGVKEY', 'DA']).first().reset_index(drop=False)
    print("shape after removing same-day multi events:", sdc_tnic_one.shape)
    sdc_tnic_one.sort_values(by = ['DA'], axis=0, inplace=True)
    return sdc_tnic_one

def N01_normalize(df):
    '''
    df could be df or array
    '''
    df = df.copy()
    normalized_df=(df-df.mean())/df.std()
    return normalized_df

def minmax_normalize(df):
    df = df.copy()
    normalized_df=(df-df.min())/(df.max()-df.min())
    return normalized_df

#@dask.delayed
def dataloader_preproceser(focal_gvkey):
    '''
    WARNING: this function requires some global variables that does not explicitly written in argument.
            so the location of this function in the code cannot be changed.
    
    '''
    
    def get_arr_c(focal_gvkey):
    # part 1, get c  
        def get_focal_df(focal_gvkey):
            '''
            output: will be a df contains 3 columns: DATE, AGVKEY, EVENT_TYPE, SCORE. TGVKEY
                DATE: datetime.dt object
                AGVKEY: str: 4 - 6 digits
                EVENT_TYPE: 1:self 0:peer (integer)
                SCORE: TNIC similarity last year for event type 0, otherwise 1
            
            Use DA!!!! not DE

            '''
            def helper1(row):
                if row.AGVKEY == focal_gvkey:
                    return 1 # integer 1
                else:
                    return 0 # integer 0   
            sdc_lst = []
            for focal_year in range(s_year-1, e_year):  
                with open(tmp_data_path+f"/a5_top_10_peers_tnic2_{focal_year}.pickle", 'rb') as f:
                    top_peers = pickle.load(f)
                
                

                top_peers = top_peers[focal_gvkey] # a dataframe
        #           print(top_peers)
                top_peers_lst = top_peers.gvkey2.tolist()
                selected_sdc_tnic = sdc_tnic[(sdc_tnic['AGVKEY'].isin(top_peers_lst + [focal_gvkey])) & (sdc_tnic.YEAR == focal_year+1) ] 
                selected_sdc_tnic.reset_index(drop=True)

                if selected_sdc_tnic.shape[0] > 0:
                    #print(selected_sdc_tnic[['DE', 'AGVKEY']] , top_peers[['gvkey2', 'score']])
                    df = selected_sdc_tnic[['DA', 'AGVKEY', 'TGVKEY']]


                    df['EVENT_TYPE'] = df.apply(helper1, axis=1)
                    #print(df)

                    score_df = top_peers[['gvkey2', 'score']]

                    df = df.merge(score_df, left_on='AGVKEY', right_on = 'gvkey2', how = 'left')
                    df = df[['DA','AGVKEY', 'EVENT_TYPE', 'score', 'TGVKEY']]
    #                print(df)
                    df = df.fillna(1)
                    df.columns = ['UPDATE_DATE','AGVKEY','EVENT_TYPE', 'SCORE', 'TGVKEY'] # rename
                    df = df.reset_index(drop=True)
                    sdc_lst.append(df)
                    #print(len(sdc_lst))
            #print(focal_gvkey, len(sdc_lst))
            assert len(sdc_lst) != 0, "The Training Data Length is too short that some frequent acquirers have no event happened in that period"
            focal_df = pd.concat(sdc_lst, axis=0) 
            focal_c = focal_df.reset_index(drop=True) 
            focal_c = focal_c.sort_values(by = ['UPDATE_DATE']) # date time is unsortable..
            focal_c.reset_index(drop=True, inplace=True)
            return focal_c

        def convert_date(df):
            def datetime_converter(date_time):
                base_time = np.datetime64('1997-01-01')
                days_diff = np.datetime64(date_time.date()) - base_time
                return days_diff.astype(int)
            for idx, row in df.iterrows():
                df.loc[idx, 'UPDATE_DATE'] = datetime_converter(df.loc[idx, 'UPDATE_DATE'])

            df.sort_values(by = ['UPDATE_DATE']).reset_index(drop=True, inplace=True)
            return df

        def making_time_diff(focal_c2):
            '''
            df = focal_c; update date is the integer form that count the date from base_date (1997 01 01)

            WARNING: the No.1 event set time-diff = 0
            '''
            tmp_columns = focal_c2.columns.tolist()
      
            focal_c2['UPDATE_DATE'] = [0] + [1 if timediff==0 else timediff for timediff in focal_c2.UPDATE_DATE.diff().tolist()[1:] ]
            focal_c2.columns = ['time_diff'] + tmp_columns[1:]
            return focal_c2

        def __main__():
            focal_c = get_focal_df(focal_gvkey)
            focal_c2 = convert_date(focal_c.copy())
            focal_c3 = making_time_diff(focal_c2.copy())
            arr_c = np.array(focal_c3[['time_diff', 'EVENT_TYPE', 'SCORE']])
            
            return arr_c, focal_c

        arr, focal_c = __main__()

        return arr, focal_c

    def get_arr_b(focal_c, focal_gvkey):
        def add_datetime(df):
            def helper(row):
                return np.datetime64(str(row.year+1)+'-01-01')
            df['UPDATE_DATE'] = df.apply(helper, axis=1)
            return df
        def obtain_fv(focal_gvkey, focal_c, fv):
            ## here I +1 since I want to use the latest fin var  of that year
            year_min, year_max = min([date.year for date in focal_c.UPDATE_DATE.tolist()]), max([date.year for date in focal_c.UPDATE_DATE.tolist()]) + 1 
            fv_subset = fv[(fv.year >= year_min-1) & (fv.year <= year_max-1) & (fv.gvkey == focal_gvkey)]
            fv_subset = fv_subset[['gvkey', 'year','UPDATE_DATE', 'at', 'sale', 'ch', 'm2b', 'lev', 'roa', 'ppe',
               'cash2asset', 'cash2sale', 'sale2asset', 'de', 'roe', 'd_sale', 'd_at']]
            fv_subset.columns=['AGVKEY', 'year','UPDATE_DATE', 'at', 'sale', 'ch', 'm2b', 'lev', 'roa', 'ppe',
               'cash2asset', 'cash2sale', 'sale2asset', 'de', 'roe', 'd_sale', 'd_at']
            
            
            return fv_subset

        def __main__():
            with open(tmp_data_path+"/afreq_full_fv.pickle", "rb") as f:
                fv = pickle.load(f)
            fv = add_datetime(fv)
            focal_b = obtain_fv("5047", focal_c, fv)
            arr_b = np.array(focal_b.iloc[:, 3:])
            arr_b = minmax_normalize(arr_b)
            return arr_b, focal_b

        arr = __main__()
        return arr

    def create_main_timeline(focal_b, focal_c):
        '''
        WARNING: GLOBAL and LOCAL time both start from 0!

        '''
        def helper(row):
            if (row.EVENT_TYPE == 1) or (row.EVENT_TYPE == 0):
                return 'past'
            else:
                return "fv"

        def helper2(row):
            if row.EVENT_TYPE == 1:
                return "1"
            elif row.EVENT_TYPE == 0:
                return "2"
            else:
                return "3"

        tmp = pd.concat([focal_c, focal_b]).sort_values(by=['UPDATE_DATE'])
        tmp['EVENT_TYPE_countcreater'] = tmp.apply(helper, axis=1)
        tmp['EVENT_TYPE_true'] = tmp.apply(helper2, axis=1)
        tmp['LOCAL_IDX'] = tmp.groupby(['EVENT_TYPE_countcreater'])['UPDATE_DATE'].rank(ascending=True) -1 # rank start with 1
        tmp['LOCAL_IDX'] = tmp['LOCAL_IDX'].astype(int)
        ## with local_idx using rank, local idx is not continuous (may have gap)
        
        tmp_columns = tmp.columns
        tmp.reset_index(drop=True, inplace=True)
        tmp.reset_index(drop=False, inplace=True)

        tmp.columns = ['GLOBAL_IDX']+ tmp_columns.tolist() # rename global index

        tmp = tmp[['GLOBAL_IDX', 'LOCAL_IDX', 'UPDATE_DATE', 'EVENT_TYPE_true', 'TGVKEY']]

        tmp.columns = ['GLOBAL_IDX', 'LOCAL_IDX', 'UPDATE_DATE', 'EVENT_TYPE', 'TGVKEY'] # rename



        return tmp
    
    def __main__():
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            arr_c, focal_c = get_arr_c(focal_gvkey)
        arr_b, focal_b = get_arr_b(focal_c, focal_gvkey)
        timeline = create_main_timeline(focal_b, focal_c)
        return arr_c, arr_b, timeline
    
    arr_c, arr_b, timeline = __main__()
    return [arr_c, arr_b, timeline]
    




################# get delta time before

def get_delta_time_previous(timelines, training_e_year):
    '''
    this function specific for:
    - training end year 
                if training end year = 2018, then the date end is 2019-01-01

    
    output: lst, each element is a scalar 
    '''
    previous_delta_times = []
    for timeline in timelines:
        last_time = timeline[timeline.EVENT_TYPE.isin(['1', '2'])].iloc[-1, 2]
        time_diff = np.datetime64(f'{training_e_year+1}-01-01') - last_time
        previous_delta_times.append(time_diff.days)
    return previous_delta_times # of all frequent acquirers

#################### get label (scalar)
# 1: happened in the following year
# 0: did not happened in the following year



def get_label(training_e_year, forward = 1):
    '''
    if training_e_year = 2018, means "if the focal firm triggered an MA event in 2019?"
    forward: how many years foward could next self event located in
    '''
    with open(data_path+f"/dataset_top10_freq5_1997_2020.pickle", "rb") as f:
        _, _, timelines = pickle.load(f) 

    # load full data
    labels = []
    for timeline in timelines:
        year_after = timeline[(timeline.UPDATE_DATE > np.datetime64(f'{training_e_year+1}-01-01')) & (timeline.UPDATE_DATE < np.datetime64(f'{training_e_year+1+forward}-01-01')) &  (timeline.EVENT_TYPE == '1')]
        if year_after.shape[0] == 0:
            labels.append(0)
        else:
            labels.append(1)
    
    return labels














######################### 


######################################## from  variables #######################
'''
the input is arr_cs, arr_bs, timelines, fv, tnic 
the output is the data described in model.py

'''
# creating helpers
def get_arr_b_idx(df):
    '''
    df is the a timeline table of a single firm (the 3rd output of preprocess function)
    output is a list
    
    
    
    '''
    sample_df = df.copy()
    sub1 = sample_df[(sample_df.EVENT_TYPE == '1')& (sample_df.LOCAL_IDX >2)] # happened at 3rd time 

    global_idxs = sub1.GLOBAL_IDX.values # array

    arr_b_idxs = []
    for global_idx in global_idxs:
        sub2 = sample_df[(sample_df.EVENT_TYPE == '3') & (sample_df.GLOBAL_IDX < global_idx)]
        arr_b_idx = sub2.iloc[-1, 1]
        arr_b_idxs.append(arr_b_idx)

    return arr_b_idxs

def get_c_t_idx(df):
    '''
    df: timeline
    '''
    sample_df = df.copy()
    sub1 = sample_df[(sample_df.EVENT_TYPE == '1')& (sample_df.LOCAL_IDX >2)] # why > 2? since we at least need 2 warmup time for c 
    local_idxs = sub1.LOCAL_IDX.values # array
    
    arr_c_idxs = []
    arr_t_idxs = []
    for local_idx in local_idxs:
        sub2 = sample_df[(sample_df.EVENT_TYPE.isin(['1','2'])) & (sample_df.LOCAL_IDX < local_idx)]
        
        # arr_c_idx and arr_t_idx is 1 row dif
        arr_c_idx = sub2.iloc[-1, 1]  # why? since the delta_t input of c is the time difference between (event-) and (event- -)
        arr_t_idx = sub2.iloc[-1, 1]
        arr_c_idxs.append(arr_c_idx)
        arr_t_idxs.append(arr_t_idx)
    return arr_c_idxs, arr_t_idxs

def convert_date(df):
    df = df.copy()
    def datetime_converter(date_time):
        base_time = np.datetime64('1997-01-01')
        days_diff = np.datetime64(date_time.date()) - base_time
        return days_diff.astype(int)
    for idx, row in df.iterrows():
        df.loc[idx, 'UPDATE_DATE_int'] = datetime_converter(df.loc[idx, 'UPDATE_DATE'])

    #df.sort_values(by = ['UPDATE_DATE']).reset_index(drop=True, inplace=True)
    return df 

def sample_negative_time_point(df, base_n_sample=10):
    '''
    df is timeline, but + 'UPDATE_DATE_int'
    
    number of negative samples is corresponding to the number of positive samples (follow the idea of negative smapling in skip-gram)
        each word, approx 10 negative samples.
    
    '''
    df = df.copy()
    max_time = df.UPDATE_DATE_int.values[-1] ### 
    sub_df = df[df.EVENT_TYPE.isin(['1', '2']) & (df.LOCAL_IDX >2)]
    min_time = sub_df.UPDATE_DATE_int.values[0]
    n_event = df[(df.EVENT_TYPE == '1') & (df.LOCAL_IDX >2)].shape[0]
    if n_event == 0:
        n_event = 1
    n_samples = base_n_sample * n_event
    samples = np.random.uniform(low=min_time, high=max_time, size=n_samples)
    return samples, max_time - min_time




def get_arr_b_idx_neg(time_samples, df):
    '''
    df is timeline + 'UPDATE_DATE_int'
    
    '''
    df = df.copy()
    df_b = df[df.EVENT_TYPE == '3']
    arr_b_idxs = []
    for time in time_samples:
        df_b_sub = df_b[df_b.UPDATE_DATE_int<time]
        arr_b_idxs.append(df_b_sub.iloc[-1, 1])
    
    return arr_b_idxs



def get_arr_c_t_idx_neg(time_samples, df):
    '''
    df is timeline + 'UPDATE_DATE_int'
    total columns are: [GLOBAL_IDX  LOCAL_IDX UPDATE_DATE EVENT_TYPE  UPDATE_DATE_int]

    time_samples, list of integers
    '''
    df = df.copy()
    df_c = df[df.EVENT_TYPE.isin(['1', '2']) & (df.LOCAL_IDX >2)]

    arr_c_idxs_neg = []
    arr_t_neg = []
    for time in time_samples:
        df_before = df_c[df_c.UPDATE_DATE_int < time]
        #print(time)
        
        arr_c_idx_neg = df_before.iloc[-1, 1] # here do not -1!
        previous_time = df_before.iloc[-1, 5]
        
        arr_c_idxs_neg.append(arr_c_idx_neg)
        #print(time, previous_time)
        arr_t_neg.append(time - previous_time)
    
    return arr_c_idxs_neg, np.array(arr_t_neg)




def get_arr_b_c_idx_i(df, s_year, e_year):
    '''
    df is timeline + 'UPDATE_DATE_int'
    total columns are: [GLOBAL_IDX  LOCAL_IDX UPDATE_DATE EVENT_TYPE  UPDATE_DATE_int]
    '''
    df = df.copy()
    # create a year variable
    def helper(row):
        return row.UPDATE_DATE.year
    df['year'] = df.apply(helper, axis=1)
    
    # qualified self event
    sub = df[(df.EVENT_TYPE == '1') & (df.LOCAL_IDX > 2)]
    
    yearly = {}
    for year in range(s_year, e_year+1):
        b_idxs = []
        c_idxs = []
        sub2 = sub[sub.year == year] # self event at particular year
        for _, row in sub2.iterrows():
            time = row.UPDATE_DATE_int # float
            # back to global df
            df_b_before = df[(df.UPDATE_DATE_int < time)&(df.EVENT_TYPE == '3')]
            df_c_before = df[(df.UPDATE_DATE_int < time)&(df.EVENT_TYPE.isin(['1','2']))]
            idx_b = df_b_before.iloc[-1, 1]
            idx_c = df_c_before.iloc[-1, 1] -1
            b_idxs.append(idx_b)
            c_idxs.append(idx_c)
            
        yearly[year] = (np.array(b_idxs), np.array(c_idxs))
    
    return yearly
        

def true_tar_idxs_i(timeline, dict_idx):
    '''
    year loop by self-event year
        TNIC related data use year-1
    
    '''

        
    # add year to timeline data
    df = timeline.copy()
    # create a year variable
    def helper(row):
        return row.UPDATE_DATE.year
    df['year'] = df.apply(helper, axis=1)
    
    # qualified self event
    sub = df[(df.EVENT_TYPE == '1') & (df.LOCAL_IDX > 2)]
    
    yearly = {}
    # loop over self-merge year
    for year in range(s_year, e_year+1):
        '''
        N_i_1 = num of candidate target
        N_i_2 = num of self event
        '''
        N_i_1 = len(gvkey_lsts[year-1]) # all target candidate in TNIC net
        b_idxs, c_idxs = dict_idx[year] # the output of ...
        N_i_2 = len(b_idxs)
        timeline_i = sub[sub.year == year] # only ith year
        targets_lst = timeline_i.TGVKEY.values.tolist() # length = N_i_2
        assert len(targets_lst) == N_i_2, "length dismatch with larget lists and N_i_2"
        idx_lst = [key_ind_maps[year-1][tgvkey] for tgvkey in targets_lst]
        #one_hot_i = (np.arange(_ == a[...,None]-1).astype(int)
        a = np.array(idx_lst)
        one_hot_i = (np.arange(N_i_1) == a[...,None]).astype(int)
        yearly[year] = one_hot_i
    return yearly



def get_node_features(fv_full, gvkey_lsts, key_ind_maps , ind_key_maps, s_year=1997, e_year=2020):
    '''
    fv_full: raw
    gvkey_lsts, key_ind_maps , ind_key_maps: raw
    
    WARNING: the output yearly's year is self-event's year!!! 
    '''
    # loop self-merge year
    yearly = {}
    for year in range(s_year, e_year+1): 
        df_gvkeys = pd.DataFrame({'gvkeys': gvkey_lsts[year-1]})
        fv_candidate = fv_full[fv_full.gvkey.isin(gvkey_lsts[year-1]) & (fv_full.year == year-1)]
        fv_i = df_gvkeys.merge(fv_candidate, left_on='gvkeys', right_on = 'gvkey', how = "left")
        fv_i.reset_index(drop=True, inplace=True)
        #print(fv_i[:5])
        arr = fv_i.iloc[:, 3:].to_numpy()
        yearly[year] = arr
        assert len(gvkey_lsts[year-1]) == arr.shape[0], "list and arr shape dismatch"
    
    return yearly

def get_net_structure(tmp_data_path, gvkey_lsts, key_ind_maps , ind_key_maps, s_year=1997, e_year=2020):
    
    yearly = {}    
    # loop over self-event year! not TNIC !
    for year in range(s_year, e_year+1):     
        with open(tmp_data_path+f'/a5_top_10_peers_tnic2_{year-1}.pickle', 'rb') as f:
            tnic = pickle.load(f)   
            df_all_lst = []
            for _,value in tnic.items():
                df_all_lst.append(value)
            df_all = pd.concat(df_all_lst)
            df_net = df_all[['gvkey1', 'gvkey2']]
            lst1 = df_net.gvkey1.values.tolist()
            lst2 = df_net.gvkey2.values.tolist()
            idx1 = [key_ind_maps[year-1][gvkey1] for gvkey1 in lst1]
            idx2 = [key_ind_maps[year-1][gvkey2] for gvkey2 in lst2]
            arr = np.array([idx1, idx2])
            assert arr.shape[0] == 2, "the dim of output is wrong"
        yearly[year] = arr
    return yearly


######## main helper ####








def get_step1_data(training_e_year, load_timeline_from_pickle, forward=1, multi_tasking=False):
    '''
    step1 data includes: arr_cs, arr_bs, timelines, previous_delta_time, labels
    
    '''
    ########### creating  arr_cs, arr_bs, timelines
    if load_timeline_from_pickle:
        with open(data_path+f"/dataset_top10_freq5_{training_s_year}_{training_e_year}.pickle", "rb") as f:
            arr_cs, arr_bs, timelines = pickle.load(f)   

    else:

        arr_cs = []
        arr_bs = []
        timelines = []
        idx_to_gvkey = {}
        results = []
        gvkey_to_idx= {}
        

        print("##### creating step 1 data #####")
        if multi_tasking:
            for i, gvkey in enumerate(a_freq_lst):
                results.append(dataloader_preproceser(gvkey))
                idx_to_gvkey[i] = gvkey

            with ProgressBar():
                results = dask.compute(results, num_workers = os.cpu_count())
            results = results[0]
            arr_cs, arr_bs, timelines = [ele[0] for ele in results], [ele[1] for ele in results], [ele[2] for ele in results]
        else:
            for i, gvkey in enumerate(tqdm(a_freq_lst[:2])):
                arr_c, arr_b, timeline = dataloader_preproceser(gvkey)
                arr_cs.append(arr_c)
                arr_bs.append(arr_b)
                timelines.append(timeline)
                idx_to_gvkey[i] = gvkey
                gvkey_to_idx[gvkey] = i

            
        #assert len(arr_cs) == len(arr_bs) == len(timelines) == len(a_freq_lst), "length of 3 outputs dismatch with len(a_freq_lst)"

        with open(data_path+f"/dataset_top10_freq5_test_{training_s_year}_{training_e_year}.pickle", "wb") as f:
            pickle.dump((arr_cs, arr_bs, timelines), f)

    ############ creating  previous_delta_time, labels
    previous_delta_time = get_delta_time_previous(timelines, training_e_year)
    labels = get_label(training_e_year, forward=forward)
    
    return arr_cs, arr_bs, timelines, np.array(previous_delta_time), np.array(labels)



def create_dataset(arr_cs, arr_bs, timelines, previous_delta_time, labels):
    ma_dataset = []
    print("##### creating step 2 data #####")
    for i, gvkey in enumerate(tqdm(a_freq_lst[:2])):
        '''
        the rest variables are for a specific freq acquirer
        
        Output:
            ma_dataset is a list
                all elements are np-arrays
        '''
        
        
    
        ######### get arr_c and arr_delta_time
        arr_c = arr_cs[i]
        #arr_c = arr_c[1:] # remove the first row! Since there's no "delta_t" data for the first self/peer event
        arr_delta_time = arr_c[1:, 0] 
        # get arr_b
        arr_b = arr_bs[i]
        
        ######## Event data
        timeline = convert_date(timelines[i]) # add UPDATE_DATE_int column
        #print(timeline.columns)
        # arr_b_idx
        arr_b_idx = get_arr_b_idx(timeline)
        arr_b_idx = np.array(arr_b_idx)
        # arr_c_idx and arr_delta_time
        arr_c_idx, _ = get_c_t_idx(timeline)

        # arr_t_idx is the same as arr_c_idx
        arr_c_idx = np.array(arr_c_idx)
        arr_t_idx = arr_c_idx

        arr_delta_time = arr_delta_time[list(arr_t_idx)]

        event_data = (arr_b_idx, arr_c_idx, arr_delta_time) #############
        
        ######## Non- Event data
        # estimate_time_length
        samples, estimate_time_length = sample_negative_time_point(timeline)
        #  arr_b_idx
        non_arr_b_idx =  get_arr_b_idx_neg(samples, timeline)
        non_arr_b_idx = np.array(non_arr_b_idx)

        non_arr_c_idx, non_arr_delta_time = get_arr_c_t_idx_neg(samples, timeline)
        non_arr_c_idx = np.array(non_arr_c_idx)

        non_event_data = (non_arr_b_idx, non_arr_c_idx, non_arr_delta_time) #############
        ######## Choice data dict
        dict_idx = get_arr_b_c_idx_i(timeline, s_year, e_year)
        true_tar_idxs = true_tar_idxs_i(timeline, dict_idx)
        node_feature = get_node_features(fv_full, gvkey_lsts, key_ind_maps , ind_key_maps, s_year, e_year)
        net_structure = get_net_structure(tmp_data_path, gvkey_lsts, key_ind_maps , ind_key_maps, s_year, e_year)
        choice_data_dict = (dict_idx, true_tar_idxs, node_feature, net_structure)
        ma_dataset.append((arr_b, arr_c, arr_delta_time, event_data, non_event_data, estimate_time_length, choice_data_dict, previous_delta_time, labels))
  
    return ma_dataset



















































































############ Global Variables

tmp_data_path = '../MA_data/data/tmp'
data_path = '../MA_data/data'





########## loading pre-prepared dataset
# YEAR invariant
# WARNING: self event at year "y" will use tnic and fv data at year "y-1"
sdc_tnic = pd.read_pickle(tmp_data_path+f"/sdc_tnic_1997_2020") # always load full dataset
sdc_tnic = same_day_only_one(sdc_tnic) 

# TNIC structure
with open(tmp_data_path+f"/tnic_info_3_pairs_{1997-1}_{2020-1}", 'rb') as f: # read full
    gvkey_lsts, key_ind_maps , ind_key_maps = pickle.load(f)

# frequent acquirer 
with open(data_path+f"/freq_a_info_1997_2020.pickle", "rb") as f: 
    A_freq, a_freq_lst, a_freq_idx_to_gvkey_mapping, a_freq_gvkey_to_idx_mapping = pickle.load(f)


# load full fv raw and normalize it
with open(data_path+"/fv_raw_full_1996_2019.pickle", 'rb') as f:
    fv_full = pickle.load(f)
fv_full = pd.concat([fv_full.iloc[:,:2], minmax_normalize(fv_full.iloc[:, 2:])], axis=1)



############
load_timeline_from_pickle = False

############ creating, year variant
training_s_year = 1997
s_year = training_s_year
#training_e_year = 2018 # fitting data til y-12-31


for training_e_year  in tqdm([2018]):
    e_year = training_e_year
    testing_s_year = training_e_year
    testing_e_year = testing_s_year+1
    forward= 1

        
    sdc_tnic = sdc_tnic[(sdc_tnic.YEAR <= training_e_year)] 


    # create arr_cs, arr_bs, timelines, previous_delta_time, labels
    arr_cs, arr_bs, timelines, previous_delta_time, labels = get_step1_data(training_e_year, load_timeline_from_pickle, forward=forward)
    ma_dataset = create_dataset(arr_cs, arr_bs, timelines, previous_delta_time, labels)
    with open(data_path+f"/ma_dataset_test_{training_e_year}_{forward}.pickle", 'wb') as f:
        pickle.dump(ma_dataset, f)



############################# so far, we have arr_cs, arr_bs, timelines, previous_delta_time, labels #############

    
    



shape before removing same-day multi events: (9661, 50)
shape after removing same-day multi events: (9448, 50)


  0%|          | 0/1 [00:00<?, ?it/s]

##### creating step 1 data #####



100%|██████████| 2/2 [00:05<00:00,  2.74s/it]


##### creating step 2 data #####



100%|██████████| 1/1 [00:16<00:00, 16.31s/it]


# check data

1. check shape
2. check value

In [34]:
non_arr_b_idx, non_arr_c_idx, non_arr_delta_time = non_event_data
arr_b_idx,arr_c_idx,arr_delta_time = event_data


In [36]:
non_arr_b_idx.shape

(980,)

In [37]:
non_arr_c_idx.shape

(980,)

In [38]:
non_arr_delta_time.shape

(980,)

In [39]:
non_arr_b_idx.max()

21

In [40]:
arr_b.shape

(23, 14)

In [41]:
non_arr_c_idx.max()

193

In [42]:
arr_c.shape

(194, 3)

In [ ]:
arr_

In [33]:
arr_c[arr_c_idx[0]]

array([7., 1., 1.])

In [30]:
arr_b_idx[0]

0

In [29]:
arr_delta_time[0]

46.0

In [44]:
previous_delta_time

array([209,  32])